<a href="https://colab.research.google.com/github/TamaraEstrada/ml-predicting-house-prices/blob/main/new445proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
!pip install category_encoders
import pandas as pd
# Read the CSV file
df = pd.read_csv('Phila_housing_data 2.csv', low_memory=False)

In [165]:
df.head()
print(df.shape)
print(df.columns)

(582942, 82)
Index(['the_geom', 'the_geom_webmercator', 'assessment_date', 'basements',
       'beginning_point', 'book_and_page', 'building_code',
       'building_code_description', 'category_code',
       'category_code_description', 'census_tract', 'central_air',
       'cross_reference', 'date_exterior_condition', 'depth',
       'exempt_building', 'exempt_land', 'exterior_condition', 'fireplaces',
       'frontage', 'fuel', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'homestead_exemption',
       'house_extension', 'house_number', 'interior_condition', 'location',
       'mailing_address_1', 'mailing_address_2', 'mailing_care_of',
       'mailing_city_state', 'mailing_street', 'mailing_zip', 'market_value',
       'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms',
       'number_of_rooms', 'number_stories', 'off_street_open',
       'other_building', 'owner_1', 'owner_2', 'parcel_number', 'parcel_shape',
       'quality_grade',

In [166]:
# Dropping data that is not really necesary for prediction
# A lot of data dropped is admistrative so it will not have a direct impact on the market value
drop_column_list = ['the_geom', 'assessment_date','beginning_point', 'book_and_page', 'category_code_description', 'cross_reference',
                                        'house_number','location', 'mailing_address_1', 'mailing_address_2', 'mailing_care_of', 'mailing_city_state',
                                        'mailing_street', 'market_value_date','the_geom_webmercator','other_building','owner_1', 'owner_2', 'parcel_number',
                                        'recording_date', 'sale_date', 'registry_number', 'sale_price', 'unit', 'objectid','building_code', 'census_tract',
                                        'date_exterior_condition', 'year_built_estimate', 'house_extension', 'mailing_zip', 'sewer', 'site_type','state_code',
                                        'street_designation', 'street_name', 'street_direction', 'geographic_ward', 'pin', 'unfinished', 'utility', 'quality_grade',
                                        'separate_utilities', 'suffix', 'number_of_rooms', 'zoning', 'building_code_description_new', 'number_stories', 'garage_type',
                                        'interior_condition', 'building_code_new', 'building_code_description']

# Dont have to take care of NaN variables in market price bc it is out target value
# also theirs enough data
data = df.drop(drop_column_list, axis = 1)
data = data.dropna(subset=['market_value'])
data = data.drop(data[data.market_value == 0].index)
data.shape

(582717, 30)




```
# category_breakdown = data['category_code'].value_counts()
# # Display the breakdown
# print("Category Breakdown:")
# print(category_breakdown)
'''
‘mixed-use’ houses may also contain commercial/office property, which tend to be more expensive.
1: single family
2: multifamily
3: mixed use
4: commercial
5: industrial
6: vacant land
7: garage commercial
8: garage residenital
9: hotel
10: offices
11: special purpose
12: vacant land/non residenital
13: vacant land residential
14: appt > 4 units
15: retail
16: blanco
```


In [167]:
data = data.drop(data[data['category_code'] == 3].index)
data = data.drop(data[data['category_code'] == 4].index)
data = data.drop(data[data['category_code'] == 5].index)
data = data.drop(data[data['category_code'] == 6].index)
data = data.drop(data[data['category_code'] == 7].index)
data = data.drop(data[data['category_code'] == 8].index)
data = data.drop(data[data['category_code'] == 9].index)
data = data.drop(data[data['category_code'] == 10].index)
data = data.drop(data[data['category_code'] == 11].index)
data = data.drop(data[data['category_code'] == 12].index)
data = data.drop(data[data['category_code'] == 13].index)
data = data.drop(data[data['category_code'] == 14].index)
data = data.drop(data[data['category_code'] == 15].index)
data = data.drop(data[data['category_code'] == 16].index)

```
# dealing with basement NaNs and zero

# NaN    169582  my definition)
# D = Full – Unknown Finish
# F = Partial - Semi-Finished
# H = Partial - Unknown Finish
# C = Full - Unfinished
# A = Full - Finished
# J = Unknown Size - Unfinished
# 0 = No Basement [NOTE: REPLACING '0' WITH 'K']
# E = Partial - Finished
# I = Unknown Size - Finished
# G = Partial - Unfinished
# B = Full - Semi-Finished

```




In [168]:
data.loc[:, "basements"] = data.loc[:, "basements"].fillna('L') # NaN
data.loc[:, "basements"] = data['basements'].replace('0', 'K') # 0

# dropped these bc they dont have a definition
data = data.drop(data[data.basements == 1].index)
data = data.drop(data[data.basements == 2].index)
data = data.drop(data[data.basements == 3].index)
data = data.drop(data[data.basements == 4].index)

In [169]:
# dealing with central_air NaN and 0
data.loc[:, "central_air"] = data['central_air'].replace('0', 'N')
data.loc[:, "central_air"] = data['central_air'].fillna('N')
# replace 1 with yes
data.loc[:, "central_air"] = data['central_air'].replace('1', 'Y')


In [170]:
# Dropping NaN values bc it has a high impact on the price so
data = data.dropna(subset=['exterior_condition'])


```


# dealing with exterior_condition NaN and 0
# 4.0    392834
# 3.0     48507
# 5.0     24331
# 1.0     16186
# 2.0     12542
# 7.0      3507
# 6.0      2665
# 0.0        62

# 4.0 = Average
# 3.0 = Above Average
# 5.0 = Below Average
# 2.0 = Newer Construction / Rehabbed
# 1.0 = Newest Construction
# 7.0 = Sealed / Structurally Compromised
# 6.0 = Vacant
### 0.0 = Not Applicable


```







```
# Fuel -- heating fuel
# A = Natural Gas
# C = Electric
# B = Oil heat
# E = Solar
# G = Other
# H = None
```

In [171]:
# dropping 'I' bc it will skew
data = data.drop(data[data.fuel == 'I'].index)
data = data.drop(data[data.fuel == 'D'].index)
data.loc[:, "fuel"] = data.loc[:, "fuel"].fillna('G') # NaN

In [172]:
# deaing with NaN and lat and lng
# capture the geographic location of properties, which can significantly influence their market values
# The decision to take the absolute value of longitude in this context is a modeling
# choice based on how longitude is being used as a feature in the machine learning model
# Magnitude of longitude, which simplifies the relationship between location and target variable.

# Filling missing values with the mean of that 'lng' data

data["lng"] = data.lng.fillna(data.lng.mean())
# convert negative values to negative
data.loc[:, "lng"] = data['lng'].abs()

# Filling missing values with the mean of that 'lat' data
data["lat"] = data.lat.fillna(data.lat.mean())

In [173]:
# Dealing with garage spaces
# NaN is given to 0 group
data.loc[:, "garage_spaces"] = data['garage_spaces'].fillna(0.0)

```
# A = Common brick
# B = Brick and Siding
# E = Stone
# C = Frame and siding
# F = Stucco/Cement
# G = Other/Mix
# H = Brick and stucco
# J = Stone/stucco
# I = Stucco and siding
# D = Frame and shingle
```



In [174]:
# General construction NaNs and 0's

# Dropping NaN
data = data.dropna(subset=['general_construction'])
data.shape

(488247, 30)

In [175]:
# parcel_shape

# Replacing the "NaN's" with 'F'
data.loc[:, "parcel_shape"] = data['parcel_shape'].fillna('F')

# E = Rectangular
# A = Other than a square, rectangular or triangle
# B = Curves, narrow portions, long access driveways
# C = Triangular (711)
# F = NaN (141)
# D = Long and narrow

In [176]:
# dropping subset
data = data.dropna(subset=['topography'])

In [177]:
# adding NaN to N categoruy has NaN in it
data.loc[:, "type_heater"] = data['type_heater'].fillna('N')
# data.type_heater.value_counts(dropna=False)

<ipython-input-177-3f2f60d81540>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "type_heater"] = data['type_heater'].fillna('N')


```
# I = Typical/Other and NaNs
# A = Cityscape / Skyline
# C = Park / Green Area
# N = Not Applicable
# D = Commercial
# H = Industrial
# E = Edifice / Landmark
# B = Flowing Water

# 0, c, a, b, i, d, h, e

```




In [178]:
# view_type
data = data.dropna(subset=['view_type'])

In [179]:
# dropping bc NaN is not a lot but having a fireplace affects the market price by alot
data = data.dropna(subset=['fireplaces'])

In [180]:
# dropping NaN number of bedrooms and bathrooms
print(data.number_of_bedrooms.value_counts(dropna=False))
data = data.dropna(subset=['number_of_bedrooms', 'number_of_bathrooms'])
data.shape

3.0     281590
0.0      90093
4.0      48422
2.0      28519
5.0       5723
6.0       3078
1.0       1204
7.0        633
8.0        473
9.0        196
12.0        60
10.0        55
NaN         42
11.0        33
13.0        13
14.0        10
31.0        10
30.0         5
15.0         4
44.0         3
16.0         2
18.0         2
40.0         1
45.0         1
Name: number_of_bedrooms, dtype: int64


(458578, 30)

In [181]:
data = data.dropna(subset=['depth'])

In [182]:
data = data.dropna(subset=['exterior_condition'])

In [183]:
data = data.dropna(subset=['zip_code'])

In [184]:
data = data.dropna(subset=['off_street_open'])

In [185]:
data = data.dropna(subset=['taxable_building'])

In [186]:
# Check for NaN values in the entire DataFrame
nan_values = data.isna().sum()
print("NaN values in the DataFrame:")
print(nan_values)

NaN values in the DataFrame:
basements               0
category_code           0
central_air             0
depth                   0
exempt_building         0
exempt_land             0
exterior_condition      0
fireplaces              0
frontage                0
fuel                    0
garage_spaces           0
general_construction    0
homestead_exemption     0
market_value            0
number_of_bathrooms     0
number_of_bedrooms      0
off_street_open         0
parcel_shape            0
street_code             0
taxable_building        0
taxable_land            0
topography              0
total_area              0
total_livable_area      0
type_heater             0
view_type               0
year_built              0
zip_code                0
lat                     0
lng                     0
dtype: int64


In [187]:
print(data.columns)
data

Index(['basements', 'category_code', 'central_air', 'depth', 'exempt_building',
       'exempt_land', 'exterior_condition', 'fireplaces', 'frontage', 'fuel',
       'garage_spaces', 'general_construction', 'homestead_exemption',
       'market_value', 'number_of_bathrooms', 'number_of_bedrooms',
       'off_street_open', 'parcel_shape', 'street_code', 'taxable_building',
       'taxable_land', 'topography', 'total_area', 'total_livable_area',
       'type_heater', 'view_type', 'year_built', 'zip_code', 'lat', 'lng'],
      dtype='object')


,basements,category_code,central_air,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,fuel,...,taxable_land,topography,total_area,total_livable_area,type_heater,view_type,year_built,zip_code,lat,lng
11,D,1,N,64.0,0.0,0.0,4,0.0,14.0,G,...,2600.0,F,776.0,258.0,H,I,1899.0,19125.0,39.977132,75.127125
18,L,2,N,100.0,329760.0,82440.0,4,0.0,16.0,G,...,0.0,F,1600.0,2540.0,N,I,1890.0,19121.0,39.979887,75.160682
24,D,2,N,78.0,311700.0,0.0,3,0.0,16.0,G,...,116100.0,F,1244.0,1280.0,H,I,1935.0,19130.0,39.971723,75.181206
27,L,1,N,44.0,32000.0,8000.0,4,0.0,14.0,G,...,0.0,F,609.0,840.0,N,I,1915.0,19132.0,39.991279,75.174171
115,D,1,N,57.0,17700.0,5900.0,4,0.0,15.0,G,...,0.0,F,869.0,1619.0,H,A,1920.0,19130.0,39.966744,75.167490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582922,L,1,N,57.0,21520.0,5380.0,3,0.0,15.0,G,...,0.0,F,860.0,1036.0,N,I,1920.0,19139.0,39.962816,75.228366
582923,L,1,N,78.0,0.0,0.0,4,0.0,15.0,G,...,9680.0,F,1170.0,1165.0,N,I,1925.0,19131.0,39.970289,75.230985
582924,L,1,N,43.0,80000.0,0.0,4,0.0,16.0,G,...,53020.0,A,688.0,960.0,H,I,1920.0,19147.0,39.933456,75.154881
582925,D,1,N,53.0,0.0,0.0,4,0.0,14.0,G,...,39880.0,F,742.0,1096.0,H,I,1923.0,19145.0,39.928696,75.172830


Now can begin Feature Engineering

In [188]:
print(data.basements.value_counts(dropna=False))
print(data.category_code.value_counts(dropna=False))
print(data.central_air.value_counts(dropna=False))
print(data.view_type.value_counts(dropna=False))
print(data.depth.value_counts(dropna=False))
print(data.exempt_building.value_counts(dropna=False))
print(data.exempt_land.value_counts(dropna=False))
print(data.exterior_condition.value_counts(dropna=False))
print(data.fireplaces.value_counts(dropna=False))
print(data.frontage.value_counts(dropna=False))
print(data.fuel.value_counts(dropna=False))
print(data.garage_spaces.value_counts(dropna=False))
print(data.general_construction.value_counts(dropna=False))
print(data.homestead_exemption.value_counts(dropna=False))
print(data.number_of_bathrooms.value_counts(dropna=False))
print(data.number_of_bedrooms.value_counts(dropna=False))
print(data.off_street_open.value_counts(dropna=False))
print(data.parcel_shape.value_counts(dropna=False))
print(data.street_code.value_counts(dropna=False))
print(data.taxable_building.value_counts(dropna=False))
print(data.taxable_land.value_counts(dropna=False))
print(data.topography.value_counts(dropna=False))
print(data.total_area.value_counts(dropna=False))
print(data.total_livable_area.value_counts(dropna=False))
print(data.type_heater.value_counts(dropna=False))
print(data.year_built.value_counts(dropna=False))
print(data.zip_code.value_counts(dropna=False))
print(data.lat.value_counts(dropna=False))
print(data.lng.value_counts(dropna=False))


L    135395
D    106961
F     63856
H     57204
C     33419
A     24018
E      9517
K      8403
J      7516
G      4285
B      3046
I      2032
1       201
4        39
2        12
3         1
Name: basements, dtype: int64
1    420045
2     35860
Name: category_code, dtype: int64
N    356298
Y     99607
Name: central_air, dtype: int64
I    432171
A     10739
C      5657
H      2603
D      2465
E      1846
0       265
B       159
Name: view_type, dtype: int64
100.0    31518
90.0     19277
70.0     14941
60.0     14327
50.0     13650
         ...  
678.0        1
481.0        1
504.0        1
650.0        1
488.0        1
Name: depth, Length: 555, dtype: int64
0.0         206354
80000.0     185520
53200.0        177
58400.0        172
64400.0        170
             ...  
708400.0         1
253146.0         1
23040.0          1
219946.0         1
32596.0          1
Name: exempt_building, Length: 13472, dtype: int64
0.00         414049
14800.00        387
15600.00        373
15200.00      

```
Feature Engineering:
**Binary:**
Basement
General Construction
Fuel
Parcel Shape
Type Heater
View type
Exterior Condition
taxable land
zip code

**One Hot:**
Central Air

**Numeric:**
Category_code
Depth
exempt_building
exempt_land
fireplaces
frontage
garage spaces
homestead exepmtion
num bedrooms
num bathrooms
off street open
street code
taxable building
total area
total liveable area
year build
lat
lng
```

In [189]:
# Binary Encode Caterogic values
import category_encoders as ce
binary_encoding = ce.BinaryEncoder(cols=['basements', 'general_construction',
                                         'fuel', 'parcel_shape', 'type_heater',
                                         'view_type', 'exterior_condition',
                                         'taxable_land', 'zip_code', 'topography'])
data_fit = binary_encoding.fit_transform(data)

# One Hot Encoding
ohe_encoding = ce.OneHotEncoder(cols=['central_air'])
newest_data = ohe_encoding.fit_transform(data_fit)

newest_data

,basements_0,basements_1,basements_2,basements_3,basements_4,category_code,central_air_1,central_air_2,depth,exempt_building,...,view_type_3,year_built,zip_code_0,zip_code_1,zip_code_2,zip_code_3,zip_code_4,zip_code_5,lat,lng
11,0,0,0,0,1,1,1,0,64.0,0.0,...,1,1899.0,0,0,0,0,0,1,39.977132,75.127125
18,0,0,0,1,0,2,1,0,100.0,329760.0,...,1,1890.0,0,0,0,0,1,0,39.979887,75.160682
24,0,0,0,0,1,2,1,0,78.0,311700.0,...,1,1935.0,0,0,0,0,1,1,39.971723,75.181206
27,0,0,0,1,0,1,1,0,44.0,32000.0,...,1,1915.0,0,0,0,1,0,0,39.991279,75.174171
115,0,0,0,0,1,1,1,0,57.0,17700.0,...,0,1920.0,0,0,0,0,1,1,39.966744,75.167490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582922,0,0,0,1,0,1,1,0,57.0,21520.0,...,1,1920.0,0,1,1,0,1,0,39.962816,75.228366
582923,0,0,0,1,0,1,1,0,78.0,0.0,...,1,1925.0,0,0,0,1,1,0,39.970289,75.230985
582924,0,0,0,1,0,1,1,0,43.0,80000.0,...,1,1920.0,0,1,0,1,0,0,39.933456,75.154881
582925,0,0,0,0,1,1,1,0,53.0,0.0,...,1,1923.0,0,1,0,0,0,0,39.928696,75.172830




```
Now deal with numerial data
```




In [190]:
from sklearn.preprocessing import StandardScaler

numeric_number = ['category_code', 'depth', 'exempt_building',
       'exempt_land', 'fireplaces', 'frontage','garage_spaces', 'homestead_exemption',
       'number_of_bathrooms', 'number_of_bedrooms',
       'off_street_open', 'street_code', 'taxable_building',
       'total_area', 'total_livable_area', 'year_built', 'lat', 'lng']

# Standardizing Data
scaler = StandardScaler()
newest_data[numeric_number] = scaler.fit_transform(newest_data[numeric_number])
newest_data

# Check for outliers
z_scores = (newest_data[numeric_number] - newest_data[numeric_number].mean()) / newest_data[numeric_number].std()
print(z_scores)

# Check for outliers
threshold = 3  # Threshold for considering a value as an outlier
outliers = (z_scores.abs() > threshold).sum()
print("Number of potential outliers in each column:")
print(outliers)

# Identify rows with potential outliers
outlier_indices = (z_scores.abs() > threshold).any(axis=1)

# Remove rows with outliers
cleaned_data = newest_data[~outlier_indices]
# Check the shape of the cleaned data
print("Shape of cleaned data:", cleaned_data.shape)

cleaned_data

        category_code     depth  exempt_building  exempt_land  fireplaces  \
11          -0.292184 -0.428087        -0.656753    -0.204687   -0.152491   
18           3.422491  0.365694         3.591684    12.460947   -0.152491   
24           3.422491 -0.119394         3.359009    -0.204687   -0.152491   
27          -0.292184 -0.869076        -0.244484     1.024390   -0.152491   
115         -0.292184 -0.582433        -0.428717     0.701757   -0.152491   
...               ...       ...              ...          ...         ...   
582922      -0.292184 -0.582433        -0.379502     0.621867   -0.152491   
582923      -0.292184 -0.119394        -0.656753    -0.204687   -0.152491   
582924      -0.292184 -0.891126         0.373920    -0.204687   -0.152491   
582925      -0.292184 -0.670631        -0.656753    -0.204687   -0.152491   
582926      -0.292184 -0.295790        -0.656753    -0.204687   -0.152491   

        frontage  garage_spaces  homestead_exemption  number_of_bathrooms  

,basements_0,basements_1,basements_2,basements_3,basements_4,category_code,central_air_1,central_air_2,depth,exempt_building,...,view_type_3,year_built,zip_code_0,zip_code_1,zip_code_2,zip_code_3,zip_code_4,zip_code_5,lat,lng
11,0,0,0,0,1,-0.292185,1,0,-0.428087,-0.656754,...,1,-1.043433,0,0,0,0,0,1,-0.507324,-0.206122
27,0,0,0,1,0,-0.292185,1,0,-0.869077,-0.244484,...,1,-0.580264,0,0,0,1,0,0,-0.235057,0.480036
115,0,0,0,0,1,-0.292185,1,0,-0.582434,-0.428717,...,0,-0.435524,0,0,0,0,1,1,-0.707233,0.382593
249,0,0,0,1,0,-0.292185,1,0,-0.516285,0.096669,...,1,-0.580264,0,0,0,1,0,0,-0.263520,0.275840
443,0,0,0,1,0,-0.292185,1,0,-0.538335,-0.656754,...,1,-0.580264,0,0,0,1,0,0,-0.258647,0.268179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582922,0,0,0,1,0,-0.292185,1,0,-0.582434,-0.379503,...,1,-0.435524,0,1,1,0,1,0,-0.782825,1.270483
582923,0,0,0,1,0,-0.292185,1,0,-0.119395,-0.656754,...,1,-0.290783,0,0,0,1,1,0,-0.639017,1.308672
582924,0,0,0,1,0,-0.292185,1,0,-0.891127,0.373921,...,1,-0.435524,0,1,0,1,0,0,-1.347874,0.198693
582925,0,0,0,0,1,-0.292185,1,0,-0.670632,-0.656754,...,1,-0.348679,0,1,0,0,0,0,-1.439482,0.460485


```
Model Development
Using the preprocessed and engineered data, students will develop machine learning models to predict house prices. They will experiment with different algorithms such as linear regression, decision trees, random forests, or gradient boosting, and select the best-performing model based on evaluation metrics.
```



In [191]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression

X = cleaned_data.drop('market_value', axis=1)
y = cleaned_data['market_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)

# Train the model
dt_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred_dt = dt_regressor.predict(X_test)

# Perform PCA with different numbers of principal components
num_components = [20, 30, 40, 50]  # Adjust as needed
pca_results = {}

for n in num_components:
    pca = PCA(n_components=n)

    # Fit PCA to the training data
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Linear regression on the new data
    lr_pca = LinearRegression()
    lr_pca.fit(X_train_pca, y_train)
    y_pred_pca = lr_pca.predict(X_test_pca)


    # Mean squared error for RMSE
    mse_pca = mean_squared_error(y_test, y_pred_pca)
    pca_results[n] = mse_pca
    # Calculate RMSE
    rmse_pca = mse_pca ** 0.5


    # Calculate R^2
    r_squared_pca = r2_score(y_test, y_pred_pca)
    print(f"\nPCA with {n} components:")
    print("RMSE:", rmse_pca)
    print("R^2:", r_squared_pca)

# PCA
print("\nPCA:")
for n, mse in pca_results.items():
    print(f"Components: {n}, MSE: {mse}")

# Perform linear reg with SelectKBest
selector = SelectKBest(score_func=f_regression, k=30)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Linear Regression
lr_fs = LinearRegression()
lr_fs.fit(X_train_selected, y_train)
y_pred_fs = lr_fs.predict(X_test_selected)

# Mean squared error
mse_fs = mean_squared_error(y_test, y_pred_fs)
print("\nLinear Reg w/ Feature Selection:")
print(f"MSE: {mse_fs}")



PCA with 20 components:
RMSE: 50333.30349870457
R^2: 0.7835630861797578

PCA with 30 components:
RMSE: 46304.89224294708
R^2: 0.8168216181916818

PCA with 40 components:
RMSE: 33729.332732767725
R^2: 0.9028067819789779

PCA with 50 components:
RMSE: 4736.975389454082
R^2: 0.9980829973237555

PCA:
Components: 20, MSE: 2533441441.0927052
Components: 30, MSE: 2144143045.6309407
Components: 40, MSE: 1137667886.5977561
Components: 50, MSE: 22438935.84029365


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:308: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)



Linear Reg w/ Feature Selection:
MSE: 9337305.616066331


In [192]:
from sklearn.tree import DecisionTreeRegressor

# Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
dt_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred_dt = dt_regressor.predict(X_test)

# Evaluate the model
mse_dt = mean_squared_error(y_test, y_pred_dt)
rmse_dt = mse_dt ** 0.5
r_squared_dt = r2_score(y_test, y_pred_dt)

print("\nDecision Tree:")
print("RMSE:", rmse_dt)
print("R^2:", r_squared_dt)



Decision Tree:
RMSE: 2213.0828824919668
R^2: 0.999581576738362
